# Facefusion on Sagemaker

## build image

In [ ]:
# 在 Facefusion-Sagemaker-Studio-Lab 目录执行如下命令，如上docker file是已CPU举例的，可以修改使用GPU 可以参考gpu_Dockerfile
!./build_and_push.sh faces-swap-on-sagemaker

In [2]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
image="faces-swap-on-sagemaker"
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

full_image_uri=f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image}:latest"
print(full_image_uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
687912291502.dkr.ecr.us-west-2.amazonaws.com/faces-swap-on-sagemaker:latest


## remote debug test

In [3]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'facefusion')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'facefusion')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-687912291502/facefusion/assets/model.tar.gz


In [7]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
model = Model(image_uri=full_image_uri, model_data=model_data, role=role,dependencies=[SSHModelWrapper.dependency_dir()] )

In [8]:
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
instance_type = "ml.g5.4xlarge"
endpoint_name = sagemaker.utils.name_from_base("facefusion-byoc")
#endpoint_name = "facefusion-byoc-2024-06-20-05-42-13-578"

ssh_wrapper = SSHModelWrapper.create(model, connection_wait_time_seconds=0)  # <--NEW--

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    wait=True
)

-------------!

In [ ]:
instance_ids = ssh_wrapper.get_instance_ids(timeout_in_sec=100)  # <--NEW-- 
print(f"To connect over SSM run: aws ssm start-session --target {instance_ids[0]}")

## SM endpoint test

### create sagemaker model

In [ ]:
import boto3
import re
import os
import json
import uuid
import boto3
import sagemaker
from time import gmtime, strftime
## for debug only
from sagemaker_ssh_helper.wrapper import SSHModelWrapper
sm_client = boto3.client(service_name='sagemaker')



def create_model():
    image=full_image_uri
    model_name="facefusion-sagemaker-01"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    create_model_response = sm_client.create_model(
        ModelName=model_name,
        ExecutionRoleArn=role,
        Containers=[{"Image": image}],
    )
    print(create_model_response)
    return model_name

In [ ]:
model_name=create_model()

### create endpoint configuration

In [ ]:
endpointConfigName = "facefusion-sagemaker-configuration"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint_configuration():
    create_endpoint_config_response = sm_client.create_endpoint_config(     
        EndpointConfigName=endpointConfigName,
        ProductionVariants=[
            {
                "ModelName":"facefusion-sagemaker-012024-03-28-04-00-03",
                #"ModelName":model_name,
                "VariantName": "facefusion-sagemaker"+"-variant",
                "InstanceType": "ml.g5.2xlarge",  # 指定 g5.2xlarge 机器
                "InitialInstanceCount": 1,
                "ModelDataDownloadTimeoutInSeconds": 1200,
                "ContainerStartupHealthCheckTimeoutInSeconds": 1200
            }
        ],
    )
    print(create_endpoint_config_response)
    return endpointConfigName

In [ ]:
create_endpoint_configuration()

### create endpoint

In [ ]:
endpointName="facefusion-sagemaker-endpoint"+strftime("%Y-%m-%d-%H-%M-%S", gmtime())
def create_endpoint():
    create_endpoint_response = sm_client.create_endpoint(
        EndpointName=endpointName,
        #EndpointConfigName="facefusion-sagemaker-configuration2024-03-28-04-03-53",
        EndpointConfigName=endpointConfigName
    )
    print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])
    resp = sm_client.describe_endpoint(EndpointName=endpointName)
    print("Endpoint Status: " + resp["EndpointStatus"])
    print("Waiting for {} endpoint to be in service".format("facefusion-sagemaker-endpoint"))
    waiter = sm_client.get_waiter("endpoint_in_service")
    waiter.wait(EndpointName=endpointName)

In [ ]:
create_endpoint()

## Realtime inferecne with sagemaker endpoint

In [ ]:
import json
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")
#endpointName="facefusion-sagemaker-endpoint2024-04-03-23-49-44"
request = {"method":"submit","input":['-s','s3://sagemaker-us-west-2-687912291502/images/image1.jpg',
                                      '-t','s3://sagemaker-us-west-2-687912291502/video/test.mp4',
                                      '-o','/tmp/','-u','s3://sagemaker-us-west-2-687912291502/video/test_out2.mp4',
                                      '--headless'],}
def invoke_endpoint():
    content_type = "application/json"
    request_body = request
    payload = json.dumps(request_body)
    print(payload)
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpointName,
        ContentType=content_type,
        Body=payload,
    )
    result = response['Body'].read().decode()
    print('返回：',result)

In [ ]:
response=invoke_endpoint()

In [ ]:
!aws s3 cp s3://sagemaker-us-west-2-687912291502/video/test_out2.mp4 ./

## Async inference

In [ ]:
_time_tag = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
_variant_name =  'facusion-'+ _time_tag
endpoint_config_name = f'facefusion-{str(uuid.uuid4())}'

response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': _variant_name,
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.c5.large',
            'InitialVariantWeight': 1
        },
    ]
    ,
    AsyncInferenceConfig={
        'OutputConfig': {
            'S3OutputPath': f's3://{bucket}/stablediffusion/asyncinvoke/out/'
        }
    }
)

In [ ]:
endpoint_name = f'facefusion-{str(uuid.uuid4())}'

response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName="endpoint_config_name",
    
)

print(f'终端节点:{endpoint_name} 正在创建中，首次启动中会加载模型，请耐心等待, 请在控制台上查看状态')


In [ ]:
import time
def predict_async(endpoint_name,payload):
    runtime_client = boto3.client('runtime.sagemaker')
    input_file=str(uuid.uuid4())+".json"
    s3_resource = boto3.resource('s3')
    s3_object = s3_resource.Object(bucket, f'stablediffusion/asyncinvoke/input/{input_file}')
    payload_data = json.dumps(payload).encode('utf-8')
    s3_object.put( Body=bytes(payload_data))
    input_location=f's3://{bucket}/stablediffusion/asyncinvoke/input/{input_file}'
    print(f'input_location: {input_location}')
    response = runtime_client.invoke_endpoint_async(
        EndpointName=endpoint_name,
        InputLocation=input_location
    )
    result =response.get("OutputLocation",'')
    wait_async_result(result)
    
def wait_async_result(output_location,timeout=60):
    current_time=0
    while current_time<timeout:
        if s3_object_exists(output_location):
            print("have async result")
            draw_image(output_location)
            break
        else:
            time.sleep(5)
def s3_object_exists(s3_path):
    """
    s3_object_exists
    """
    try:
        s3 = boto3.client('s3')
        base_name=os.path.basename(s3_path)
        _,ext_name=os.path.splitext(base_name)
        bucket,key=get_bucket_and_key(s3_path)
        
        s3.head_object(Bucket=bucket, Key=key)
        return True
    except Exception as ex:
        print("job is not completed, waiting...")   
        return False

## client lib test

In [22]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/images/

2024-04-16 01:54:56     124941 image1.jpg
2024-06-15 11:38:50     123139 musk.jpg
2024-06-21 08:06:06      47987 old.jpg
2024-06-15 11:40:20      15280 tangyan.jpg


In [4]:
from ModelClient import ModelClient
client = ModelClient("facefusion-v2.6")
client.set_endpoint("facefusion-byoc-2024-06-20-05-42-13-578")
job_id=client.submit_job("test03",swap_face_image_s3_path="s3://facefusiondemo/42ddd7de-6d02-41a7-9268-c931a91be022_75725c7312cc532dc379f115db8b2c69.jpeg",
                           source_video_s3_path='s3://facefusiondemo/de4291fe-abef-4f9d-ba75-b8ee9ab5fd67_mmexport1720427776894[1].mp4',
                           output_video_s3_dir='s3://facefusiondemo/out')




{"method": "submit", "input": ["-s", "s3://facefusiondemo/42ddd7de-6d02-41a7-9268-c931a91be022_75725c7312cc532dc379f115db8b2c69.jpeg", "-t", "s3://facefusiondemo/de4291fe-abef-4f9d-ba75-b8ee9ab5fd67_mmexport1720427776894[1].mp4", "--execution-providers", "cuda", "-o", "/opt/program/output/e2b8367fae0844f8964c25014dea9d74-20240710083736.mp4", "-u", "s3://facefusiondemo/out/e2b8367fae0844f8964c25014dea9d74-20240710083736.mp4", "--headless"]}
返回： {"message": "Command executed in background"}


In [16]:
from ModelClient import ModelClient
#job_id="65af789625ac4f579aa156c2e8017677-20240707021418"
status = client.get_status( "test03", job_id)
status

'not finished'

In [ ]:
!aws s3 ls s3://facefusiondemo/out/e2b8367fae0844f8964c25014dea9d74-20240710083736.mp4
from ModelClient import ModelClient
response = client.get_result(job_id)


In [28]:
print(type(response))

<class 'bytes'>


In [80]:
!aws s3 cp  s3://sagemaker-us-west-2-687912291502/video/out/9e08b0f6bd0447dabf71356772df0dbb-20240709075647.mp4 ./

download: s3://sagemaker-us-west-2-687912291502/video/out/9e08b0f6bd0447dabf71356772df0dbb-20240709075647.mp4 to ./9e08b0f6bd0447dabf71356772df0dbb-20240709075647.mp4


In [135]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/video/

                           PRE output/
                           PRE raw/
2024-06-21 10:07:45   19806949 09e76a560cb344568f08224f624ff505-20240621100643.mp4
2024-06-15 01:53:48   15511703 0aec03207c094720a0b31ff360e8baaa-20240615015252.mp4
2024-05-19 05:20:32   19226179 16a1750ae7804c8bb57c4a8cfdcb91c0-20240519051947.mp4
2024-04-26 00:30:46   19224455 354ec5a802794faaaf48343f3591670f-20240426002420.mp4
2024-06-20 08:33:17   13128549 3ea39874e63046259a0cf4eaa5b64cc9-20240620083251.mp4
2024-06-22 07:26:32   19806949 4c02e9ea02c149b7b9da2b5814c61cbe-20240622072539.mp4
2024-06-21 14:59:44    3322744 62d838c8e34a412da137a46914a28053-20240621145920.mp4
2024-06-20 05:58:07    7658160 74ab4bdc853d4cd6a06f7d76f8e38318-20240620055728.mp4
2024-04-26 00:18:33   19289209 7e758ed6615c4781822f529744e72017-20240426001203.mp4
2024-06-15 12:56:10    7703173 83f6fb14f31d4e90bb6d42950bd0d0e3-20240615125538.mp4
2024-06-15 03:12:55   15511703 845f73df5eef45eea15cfef861798351-20240615031159.mp4
2024-06-17 0